##Przestawienie problemu. Zacznijmy od pewnego filmu. Oczywiście przedtem trzeba zaimportować odpowiednie biblioteki

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('Z44BZeF-ovk', width=640, height=360)

No właśnie? Czy w niedziele o godzinie 17 gorzej się gra w piłkę?

Najpierw zainstalujmy niezbędne pakiety

In [4]:
import kagglehub
import pandas as pd
from pathlib import Path, PurePath


Teraz zaimportujmy dane i zróbmy pierwsze modyfikacje

In [33]:
pd.set_option('display.max_columns', 18)

path = kagglehub.dataset_download("filippomonte/premier-league-past-3-seasons")
file = PurePath(Path(path).joinpath('PremierLeagueMatches.csv'))
df_raw = pd.read_csv(file)
df_raw.columns = [x.lower() for x in df_raw.columns]
df = df_raw.dropna(subset=['result'])
print(df.columns)
cols_to_drop = ['matchday',  'attendance', 'referee', 'stadium', 'result', '*additional stats']
df.drop(columns=cols_to_drop,inplace=True)

Index(['matchday', 'date', 'time', 'home team', 'homescore', 'homexg',
       'awayscore', 'awayxg', 'away team', 'attendance', 'referee', 'stadium',
       'result', '*additional stats'],
      dtype='object')


C:\Users\mateu\AppData\Local\Temp\ipykernel_13108\3142554772.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=cols_to_drop,inplace=True)


In [34]:
print(df)

            date   time       home team  homescore  homexg  awayscore  awayxg  \
0     2022-08-05  20:00  Crystal Palace        0.0     1.2        2.0     1.0   
1     2022-08-06  12:30          Fulham        2.0     1.2        2.0     1.2   
2     2022-08-06  15:00       Tottenham        4.0     1.5        1.0     0.5   
3     2022-08-06  15:00   Newcastle Utd        2.0     1.7        0.0     0.3   
4     2022-08-06  15:00    Leeds United        2.0     0.8        1.0     1.3   
...          ...    ...             ...        ...     ...        ...     ...   
1044  2025-03-15  15:00     Southampton        1.0     1.3        2.0     0.4   
1045  2025-03-15  17:30     Bournemouth        1.0     1.6        2.0     0.8   
1046  2025-03-16  13:30         Arsenal        1.0     0.7        0.0     0.4   
1047  2025-03-16  13:30          Fulham        2.0     1.0        0.0     0.9   
1048  2025-03-16  19:00  Leicester City        0.0     1.0        3.0     0.9   

            away team  
0  